<a href="https://colab.research.google.com/github/geoffwoollard/learn_cryoem_math/blob/master/nb/healpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install healpy

In [2]:
!pip install pytorch3d

In [3]:
import numpy as np
import healpy as hp
import pytorch3d.transforms
import torch
from torch import tensor

# Resources
* https://healpy.readthedocs.io/en/latest/tutorial.html#NSIDE-and-ordering

# Eulers (theta, phi) to rotations

In [4]:
NSIDE = 1
NPIX = hp.nside2npix(NSIDE)
print(NPIX)

12


In [5]:
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))


In [6]:
np.rad2deg(theta)

array([ 48.1896851,  48.1896851,  48.1896851,  48.1896851,  90.       ,
        90.       ,  90.       ,  90.       , 131.8103149, 131.8103149,
       131.8103149, 131.8103149])

In [7]:
np.rad2deg(phi)

array([ 45., 135., 225., 315.,   0.,  90., 180., 270.,  45., 135., 225.,
       315.])

In [8]:
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX) # arange?
n_repeats = 2
psi.repeat(1,n_repeats).T

tensor([[0.0000],
        [0.5236],
        [1.0472],
        [1.5708],
        [2.0944],
        [2.6180],
        [3.1416],
        [3.6652],
        [4.1888],
        [4.7124],
        [5.2360],
        [5.7596],
        [0.0000],
        [0.5236],
        [1.0472],
        [1.5708],
        [2.0944],
        [2.6180],
        [3.1416],
        [3.6652],
        [4.1888],
        [4.7124],
        [5.2360],
        [5.7596]])

In [9]:
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles_2d.repeat(n_repeats,1)


tensor([[0.8411, 0.7854],
        [0.8411, 2.3562],
        [0.8411, 3.9270],
        [0.8411, 5.4978],
        [1.5708, 0.0000],
        [1.5708, 1.5708],
        [1.5708, 3.1416],
        [1.5708, 4.7124],
        [2.3005, 0.7854],
        [2.3005, 2.3562],
        [2.3005, 3.9270],
        [2.3005, 5.4978],
        [0.8411, 0.7854],
        [0.8411, 2.3562],
        [0.8411, 3.9270],
        [0.8411, 5.4978],
        [1.5708, 0.0000],
        [1.5708, 1.5708],
        [1.5708, 3.1416],
        [1.5708, 4.7124],
        [2.3005, 0.7854],
        [2.3005, 2.3562],
        [2.3005, 3.9270],
        [2.3005, 5.4978]], dtype=torch.float64)

In [10]:
n_repeats = NPIX
euler_angles = torch.hstack([euler_angles_2d.repeat(n_repeats,1), psi.repeat(1,n_repeats).T])

In [11]:
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')
rots.shape

torch.Size([144, 3, 3])

In [12]:
for NSIDE in [2**x for x in range(10)]:
  NPIX = hp.nside2npix(NSIDE)
  print(NSIDE,NPIX)

1 12
2 48
4 192
8 768
16 3072
32 12288
64 49152
128 196608
256 786432
512 3145728


# Benchmarking

|NSIDE|NPIX|time|
|-|-|-|
|1| 12|324 µs|
|2| 48|1.13 ms|
|4| 192|9.85 ms|
|8| 768|188 ms|
|16| 3072|3.01 s|

In [13]:
%%timeit

NSIDE = 1
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')

The slowest run took 5.59 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 324 µs per loop


In [14]:
%%timeit

NSIDE = 2
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')

1000 loops, best of 5: 1.04 ms per loop


In [15]:
%%timeit

NSIDE = 4
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')

100 loops, best of 5: 9.36 ms per loop


In [16]:
%%timeit

NSIDE = 8
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')

1 loop, best of 5: 188 ms per loop


In [17]:
%%timeit

NSIDE = 16
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')

1 loop, best of 5: 3.01 s per loop


`NSIDE = 32` is too memory intense. Perhaps can precmpute 2D rotation of first two angles, then do vectorized matrix multiplication of last angle with Euler matrix

In [ ]:
%%time

NSIDE = 32
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX))
psi = torch.linspace(0,2*np.pi*(1-1/NPIX) ,steps=NPIX)
euler_angles_2d = torch.vstack([tensor(theta), tensor(phi)]).T
euler_angles = torch.hstack([euler_angles_2d.repeat(NPIX,1), psi.repeat(1,NPIX).T])
rots = pytorch3d.transforms.euler_angles_to_matrix(euler_angles=euler_angles, convention='ZYZ')